# Choose and process a file in Python test script
<p>This Jupyter Notebook file was created to help the user find and process the Open Lines daily report from a local drive using Jupyter Lab.</p>
    
### Loading file Instructions:

1. Click on the folder image on the extreme left. 
2. Click on upload file arrow once the menu is open.
3. Select the file and click on OK.
4. Wait for the file to load.

### Start with the cell below
1. Press **Shift+Enter** to run the cell.
2. Click on the **Select** button
3. Click on the file you want to process
4. Click on **Select** again.

In [1]:
# Necessary libraries
import pandas as pd # I will need a dataframe to hold the data from the file
import numpy as np # I will need this library calculate days aging
import ipywidgets as widget # this library is to create the buttons I will need for the user
import datetime
from ipyfilechooser import FileChooser

# create a widget to find and select the sample file we want to read
fc = FileChooser()
display(fc)

FileChooser(path='C:\Users\jbustos\Documents\Python Scripts\Scheduling Dashboard\sources\done', filename='', t…

### After selecting the file
1. Press **Shift+Enter** to run the cell.
2. Type in the processing date using this format **m/d/yyyy** and press **Enter**
3. Click on **Process** or **Cancel**
4. Wait for the preview of the file to appear below
5. Check if **open_lines_processed.xlsx** is visible on the left of your screen.

In [3]:
# Print the file name and the next instructions
spath = fc.selected_path
fname = fc.selected_filename
open_lines = widget.Output()

# create the variable that will change the field names the source file.
fieldnames = ['chgReqIDx','so', 'ln', 'hl', 'material', 'descrip', 'plant', 'itemCat', 'schReqOwnr', 'mfgReqOwnr', 'customer', 
               'delvPri', 'createD', 'createT', 'complD', 'complT', 'chgReqCreateD', 'chgReqCreateT', 'chgReqCmplD', 
               'chgReqCmplT', 'createdBy', 'ofcRep', 'chgCurrMAD','soCurrMAD', 'reqMAD', 'apprMAD', 'chgCurrDelvD', 
               'reqDelvDate', 'apprDelvDate', 'currQty', 'urgent', 'escalation', 'inventoryM', 'engQuote', 'noAck',
               'invCheck', 'ownership', 'oDlrReqDelvD', 'orderRsn']

# fill all holidays in standard python format yyyy-mm-dd
holidayx = ['2022-01-17','2022-05-30','2022-07-04','2022-09-05','2022-11-24','2022-11-25','2022-12-26',
            '2023-01-02','2023-01-16']

# ask the user for the date of the file we are processing
prcdate = input('Type in the date of the file you are processing: ')
prcdate = datetime.datetime.strptime(prcdate,'%m/%d/%Y')

print('Ready to process', fname,'\nClick on Process to execute the script or Cancel to stop the process.')

# create the functions to be called for each button
def btn_cancel(x):
    print('Process was canceled by user.',fname,'was not processed')

def btn_process(arg):
    print('Reading file from',fc.selected)
    
    df = pd.read_excel(r'' + fc.selected, keep_default_na = False)
    print('File has been read. Starting to process')
    
    df.columns = fieldnames
    
    fsize = len(df)
    print('File has', fsize, 'rows before filtering.')
    df = df.loc[df['ownership'].isin(['S','M'])]
    fsize = len(df)
    print('File has', fsize, 'rows after filter was applied')
    
    # initiate the progress bar
    progress_bar = widget.IntProgress(min = 0, max = fsize, description = 'PROGRESS') 
    display(progress_bar) # display the bar
    
    for x, row in df.iterrows():
        # clear the variables that we are going to use
        owner = '' # Scheduling Owner
        rqdlv = '' # requested delivery date
        crdlv = '' # current delivery date
        apdlv = '' # approved delivery datecrtDt = '' # create date
    
        apmad = ''
        dayx = ''
        wkds = ''
        wkns = ''
        networkingdays = ''
        # assign the values to the variables
        owner = df.loc[x,'schReqOwnr']
    
        crtDt = datetime.datetime.strptime(df.loc[x,'createD'],'%m/%d/%Y') #m/%d/%Y, %Y%m%d
    
        if df.loc[x,'reqDelvDate'] != '':
            rqdlv = datetime.datetime.strptime(df.loc[x,'reqDelvDate'],'%m/%d/%Y')
        else:
            rqdlv= datetime.datetime.today()
    
        if df.loc[x,'chgCurrDelvD'] != '':
            crdlv = datetime.datetime.strptime(df.loc[x,'chgCurrDelvD'],'%m/%d/%Y')
        else:
            crdlv = datetime.datetime(2100, 1, 1)
    
        if df.loc[x,'apprDelvDate'] != '':
            apdlv = datetime.datetime.strptime(df.loc[x,'apprDelvDate'],'%m/%d/%Y')
        else:
            apdlv = datetime.datetime(2100, 1, 1)
    
        if df.loc[x,'apprMAD'] != '':
            apmad = datetime.datetime.strptime(df.loc[x,'apprMAD'],'%m/%d/%Y')
        else:
            apmad = datetime.datetime(2100, 1, 1)
    
        # determine if the request has been claimed or not by a Scheduling Coordinator
        df.at[x,'ownerCat'] = 'unclaimed' if owner == '' else 'claimed'
    
        # determine if the request is a move up or a move out
        df.at[x,'orderCat'] = 'move up' if rqdlv <= crdlv else 'move out'
    
        #calculate aging days using numpy
        networkingdays = np.busday_count(crtDt.date(), prcdate.date(), holidays = holidayx)
        df.at[x,'agingDays'] = int(networkingdays)
    
        #material impacted
        df['matImpacted'] = 'False'
    
        # determine if the request is a response or an open item
        df.at[x,'respOpen'] = 'responses' if apdlv != datetime.datetime(2100, 1, 1) else 'open items'
        progress_bar.value += 1
    
    # delete the entire change request id column, we don't need it.
    df.pop('chgReqIDx')

    print('Saving processed XL file as',spath + '/open_lines_processed.xlsx')
    df.to_excel(spath + '/open_lines_processed.xlsx', sheet_name = 'data', index = False)
    print('Processing is done.')
    
    with open_lines:
        display(df)

#create the buttons to run or cancel
prcs = widget.Button(description = 'Process', style = dict(button_color = 'lightgreen', font_weight = 'bold'))
stop = widget.Button(description = 'Cancel', style = dict(button_color = 'pink', font_weight = 'bold'))

prcs.on_click(btn_process)
stop.on_click(btn_cancel)

with open_lines:
    display(widget.HBox((prcs,stop)))
    
open_lines

Type in the date of the file you are processing: 11/16/2022
Ready to process 20221116050932.CHG_REQ_OPEN_LINES.xlsx 
Click on Process to execute the script or Cancel to stop the process.


Output()

### Final step:
1. Right-click on **open_lines_processed.xlsx** and download it to your device.